# Module 1 - Implementing and training a neural network

## Environment verification
Start by confirming you have PyTorch and TorchVision installed. If not, install it using the commands `pip install torch`.


In [8]:
import torch
import torchvision

## Question 1 - MLP evaluation

Import the example MLP:

In [9]:
from bobnet import BobNet

Create an instance of this model:

In [10]:
model1 = BobNet()

Get the training set and create a training data loader:

In [11]:
# get the training set
training_set = torchvision.datasets.MNIST("./data", train=True, download=True)

# create the training loader
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)

1.0%

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw


Run the training:

In [ ]:
# TODO: run training

Run the testing:

In [12]:
# TODO: run testing

### Questions
Explore the architecture on the script `mod1/bobnet.py`.
1. Why does the input layer have 784 input features?
2. Why does the output layer have 10 output features?

## Question 2 - CNN implementation

Head over to the `cnn.py` file and implement the following architecture:
- Convolutional layer with 6 output channels, 5x5 kernel size, stride 1 and tanh activation
- Average pooling layer with 6 output channels, 2x2 kernel size, stride 2 and tanh activation
- Convolutional layer with 16 output channels, 5x5 kernel size, stride 1 and tanh activation
- Average pooling layer with 16 output channels, 2x2 kernel size, stride 2 and tanh activation
- Convolutional layer with 120 output channels, 5x5 kernel size, stride 1 and tanh activation
- Fully-connected layer with 84 neurons and tanh activation
- Fully-connected layer with 10 neurons and softmax activation

Now, import the model:

In [13]:
from cnn import CNN

Create an instance of this model:

In [14]:
model2 = CNN()

NotImplementedError: Define the layers here!

Train the model:

In [ ]:
# TODO: run training

Test the model:

In [ ]:
# TODO: run testing